In [54]:
import sys, os
import re
import pandas as pd
import itertools, nltk, string 

positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))    
        
read_lexicon()
opinion_lexicon = positive_lexicon + negative_lexicon

In [99]:
import os
os.environ["CORENLP_HOME"] = r'C:\stanford-corenlp-full-2018-10-05'

import corenlp 
client = corenlp.CoreNLPClient()

to_be = ['be', 'am', 'is', 'are', 'was', 'were', 'been', 'being']

def get_adjective(sentence):
    text = sentence.lower()
    res = []
    try:
        pattern = '[{pos:JJ}] | [{pos:JJR}] | [{pos:JJS}]'
        matches = client.tokensregex(sentence, pattern)
        response = matches['sentences'][0]
        len_chunk = response['length']
        if len_chunk > 0:
            for index in range(0, len_chunk):
                res.append(response[str(index)]['text'])
    except:
        print('err : ', sentence)
        res = []
    return res

def get_verb(sentence):
    text = sentence.lower()
    res = []
    try:
        pattern = '[{pos:/VB.*/}]'
        matches = client.tokensregex(sentence, pattern)
        response = matches['sentences'][0]
        len_chunk = response['length']
        if len_chunk > 0:
            for index in range(0, len_chunk):
                word = response[str(index)]['text']
                if word not in to_be:
                    res.append(word)
    except:
        print('err : ', sentence)
        res = []
    return res

def get_op_word(middle, opinions):
    temp = 9999
    res = ''
    for word, index in opinions:
        jarak = abs(index - middle)
        if jarak < temp:
            res = word
            temp = jarak
    return res

In [72]:
import pandas as pd

df = pd.read_csv('Results/Categorization/16_lda2vec-partial-ner-money.csv')
# df.head()

In [149]:
import string

def midpoint(p1, p2):
    return int((p1+p2)/2)

def rem_punct(word):
    return word.translate(str.maketrans('', '', string.punctuation))

def create_file(category):
    sf = pd.DataFrame(columns = ['id','review', 'opinion', 'label'])
    for index in range(0, len(df)):
        if type(df['predict'][index]) == float:
            continue
        labels = df['label'][index].split(',')
        predicts = df['predict'][index].split('|')
        if category in labels and category in predicts:
            kata_kata = []
            tokens = df['review'][index].replace('  ', ' ').strip().lower().split(' ')
            terms = df['term'][index].split('|')
            if "neutral" in df['polarity'][index]:
#                 print('neutral')
                continue
            
            polarities = df['polarity'][index].split(',')

            new_polar = []
#             print(df['review'][index])
#             print(predicts)
#             print(terms)
            for x, label in enumerate(labels):
                if category == label:
                    new_polar.append(polarities[x])
            
            new_terms = []
            for x, predict in enumerate(predicts):
                if category == predict:
                    new_terms.append(terms[x])
            
            for term in new_terms:
                i = 0
                for token in tokens:
                    if term in token:
                        kata_kata.append(i)
                    i += 1
            middle = midpoint(min(kata_kata), max(kata_kata))
            
            opinion_pos = []
            i = 0
            for token in tokens:
                if rem_punct(token) in opinion_lexicon:
                    opinion_pos.append((rem_punct(token), i))
                i += 1
            
            opinion_adj = []
            if len(opinion_pos) == 0:
                opinion_adj = get_adjective(df['review'][index])
                i = 0
                for token in tokens:
                    for op in opinion_adj:
                        if op in token:
                            opinion_pos.append((rem_punct(token), i))
                    i += 1
                    
            
            opinion_vb = []
            if len(opinion_pos) == 0:
                opinion_vb = get_verb(df['review'][index])
                i = 0
                for token in tokens:
                    for op in opinion_vb:
                        if op in token:
                            opinion_pos.append((rem_punct(token), i))
                    i += 1
                    
            final_opinion = get_op_word(middle, opinion_pos)
            
            sf = sf.append({'id': df['id'][index], 
                    'review': df['review'][index],
                    'opinion': final_opinion,
                    'label': new_polar[0]
                   }, ignore_index=True)
            
    sf.to_csv("Results/Sentiwordnet/"+ category +".csv")
    sf.to_excel("Results/Sentiwordnet/"+ category +".xlsx")

In [151]:
import pandas as pd

df = pd.read_csv('Results/Categorization/16_lda2vec-partial-ner-money.csv')
# df.head()

In [145]:
# create_file('FOOD')

In [146]:
# create_file('AMBIENCE')

In [147]:
# create_file('SERVICE')

In [154]:
create_file('PRICES')

service was devine, oysters where a sensual as they come, and the price can't be beat!!!
['SERVICE', 'PRICES']
['service', 'price', '']
we took advanatage of the half price sushi deal on saturday so it was well worth it.
['PRICES']
['price', '']
the prix fixe menu is worth every penny and you get more than enough (both in quantity and quality).
['FOOD', 'FOOD', 'FOOD', 'PRICES', 'PRICES', 'PRICES']
['prix', 'fixe', 'menu', 'penny', 'quantity', 'quality', '']
$6 and there is much tasty food, all of it fresh and continually refilled.
['FOOD', 'PRICES']
['food', '$6']
add to that great service and great food at a reasonable price and you have yourself the beginning of a great evening.
['FOOD', 'FOOD', 'FOOD', 'FOOD', 'SERVICE', 'PRICES']
['great', 'great', 'great', 'food', 'service', 'price', '']
a large is $20, and toppings are about $3 each.
['FOOD', 'PRICES']
['toppings', '$20,']
great wine selection, gigondas is worth the price, and the house champagne is a great value.
['FOOD', 'FOOD

if it seemed possible to do so while there i would have fought my bill since my dinner portion of my meal was inedible!
['PRICES', 'FOOD', 'FOOD', 'FOOD']
['bill', 'dinner', 'portion', 'meal', '']
i have it a 4 instead of 5 because of the price (just chicken tikka masala - no bread of rice - is $25), which i would expect at a upscale indian restaurant but this place doesn't have an upscale feel.
['FOOD', 'AMBIENCE', 'PRICES', 'PRICES']
['restaurant', 'place', 'price', '$25),']
but if you're prepared to spend some $ and remember to ask if something they offer is complimentary, then this is the place to go for indian food
['FOOD', 'AMBIENCE', 'PRICES']
['food', 'place', '$']
too bad i had paid an extra $2 for the stone bowl.
['FOOD', 'FOOD', 'PRICES']
['stone', 'bowl', '$2']
my party had the bbe $29 fixe prix menu, which was such a wonderful deal since it also came with a flight of sake!
['FOOD', 'FOOD', 'FOOD', 'FOOD', 'PRICES', 'AMBIENCE', 'FOOD', 'PRICES']
['party', 'fixe', 'prix', 'm